In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/checknft_collections.csv').dropna(subset=['name'])

In [3]:
full_names_ds = df['name'].to_list()

In [4]:
from datasketch import MinHashLSHForest, MinHash
from multiprocessing import Pool


def get_minhash(name: str) -> MinHash:
    m_hash = MinHash(num_perm=128)
    for d in name.split():
        m_hash.update(d.encode('utf8'))
    return m_hash

forest = MinHashLSHForest(num_perm=128)


    

    


In [5]:
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm

with Pool() as pool:
    hash_values = list(tqdm(pool.map(get_minhash, full_names_ds),
                            total=len(full_names_ds)))

100%|██████████████████████████████| 669877/669877 [00:00<00:00, 3932091.03it/s]


In [6]:
result = []

for  idx, name, hash_value in zip(list(range(len(full_names_ds))), full_names_ds, hash_values):
    try:
        forest.add(name, hash_value)
    except ValueError:
        result += [(idx, name)]


In [7]:
import pickle

with open('checkpoints/tree_db_words_data.pt', 'wb') as f:
    f.write(pickle.dumps(forest))

In [8]:
forest.index()

In [9]:
for idx, hash_value in enumerate(tqdm_notebook(hash_values)):
    res = forest.query(hash_value, 5)
    if res:
        result += [(res, full_names_ds[idx])]

  0%|          | 0/669877 [00:00<?, ?it/s]

In [19]:
clear_result = [res for res in result if len(res) > 1]

In [27]:
result[200000]

(['HashGarage Specials Minter',
  'OSUVOX Minter',
  'Letter Letter Letter Minter',
  'HashGarage Minter',
  'BONK Minter'],
 'HashGarage Minter')

In [28]:
df

,id,name,description
0,3613240,KidsInNFT Genesis,KidsInNFT are hand drawn by Rayka who recently...
1,3613329,Over the rainbow stories,NaN
2,20506357,I AM Sophie the 1st,NaN
3,20506387,TheAbstractions,NaN
4,3613498,Logan and Laura,First NFTs of Logan and Laura. \nLogan and Lau...
...,...,...,...
681108,1951149,Chibi Monkey,A growing collection of individually customise...
681109,3937072,NyanFrndz,NyanFrndz is an NFT collections of 3333 unique...
681110,1588804,Portrait Studio by Daniel Gordon,Assembly presents *Portrait Studio* by Daniel ...
681111,1590583,Dragon Hide,An Ai curated NFT rare collection of Dragon sc...
